<a href="https://colab.research.google.com/github/thislis/RnE/blob/main/test_multi_Bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test

build : Bi-LSTM, Multi classification

In [ ]:
#google drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Mecab-ko download

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.50 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 4.7 MB/s 
     |████████████████████████████████| 465 kB 93.9 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-05 08:41:54--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bi

In [ ]:
#import modules

#test data reading and processing
import re, os
import pandas as pd
import numpy as np
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

#modeling
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import keras.backend as K

In [ ]:
#data loading

data_file_path = "/content/drive/Shareddrives/R&E/dataset"
train_file_list = os.listdir(data_file_path + '/train')
test_file_list = os.listdir(data_file_path + '/test')

train_data = pd.DataFrame(columns=['감정','문장'])
test_data = pd.DataFrame(columns=['감정','문장'])

for path in train_file_list:
  new = pd.read_csv(data_file_path + '/train/' + path)
  new.dropna()
  new.drop_duplicates(subset=['문장'], inplace=True)
  new['문장'] = new['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
  new['문장'].replace('', np.nan, inplace=True)
  new = new.reset_index(drop = True)
  train_data = pd.concat([train_data, new[:17000]], ignore_index = True)

for path in test_file_list:
  new = pd.read_csv(data_file_path + '/test/' + path)
  new.dropna()
  new.drop_duplicates(subset=['문장'], inplace=True)
  new['문장'] = new['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
  new['문장'].replace('', np.nan, inplace=True)
  new = new.reset_index(drop = True)
  test_data = pd.concat([test_data, new[:2000]], ignore_index = True)

<ipython-input-4-77b478a7ca3d>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  new['문장'] = new['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-4-77b478a7ca3d>:23: FutureWarning: The default value of regex will change from True to False in a future version.
  new['문장'] = new['문장'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [ ]:
#def stopword

stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '되', '음', '면']

In [ ]:
#def function - sentiment to int

def toint(text):
  if text == "분노":
    return 0

  elif text == "불안":
    return 1

  elif text == "당황":
    return 2

  elif text == "기쁨":
    return 3

  elif text == "상처":
    return 4

  elif text == "슬픔":
    return 5

In [ ]:
tft = train_data.sample(n=4, random_state = 1)
print(tft)

tft['감정'] = tft['감정'].apply(toint)
print(tft)

       감정                        문장  Unnamed: 0
65226  당황        열심히 회복해서 얼른 퇴원해야겠어     14799.0
65591  당황           집에서 기분 전환을 해야겠어     15193.0
70106  불안  최근에 살이 많이 쪄서 스트레스를 받고 있어      2121.0
84400  불안              건강 검진을 받아야겠어     16882.0
       감정                        문장  Unnamed: 0
65226   2        열심히 회복해서 얼른 퇴원해야겠어     14799.0
65591   2           집에서 기분 전환을 해야겠어     15193.0
70106   1  최근에 살이 많이 쪄서 스트레스를 받고 있어      2121.0
84400   1              건강 검진을 받아야겠어     16882.0


In [ ]:
#sentiment to int

train_data['감정'] = train_data['감정'].apply(toint)
test_data['감정'] = test_data['감정'].apply(toint)

print(train_data.sample(n=4, random_state = 1))
print(train_data.groupby('감정').size())
print(test_data.sample(n=4, random_state = 1))
print(test_data.groupby('감정').size())

       감정                        문장  Unnamed: 0
65226   2        열심히 회복해서 얼른 퇴원해야겠어     14799.0
65591   2           집에서 기분 전환을 해야겠어     15193.0
70106   1  최근에 살이 많이 쪄서 스트레스를 받고 있어      2121.0
84400   1              건강 검진을 받아야겠어     16882.0
감정
0    17000
1    17000
2    17000
3    17000
4    17000
5    17000
dtype: int64
       감정                                            문장  Unnamed: 0
7163    4  아직 일자리를 구하지 못했거든 친구들은 모두 취직을 했는데 왜 나만 안되는 걸까      1164.0
10385   2        친구가 일용직인데 일도 가끔만 나가고 재산도 모으지 않아서 당황스러워       386.0
1902    3                         응 그래서 앞으로도 꾸준히 할 생각이야      1911.0
3397    1                       상사분과 상담해서 업무 분담을 요청해야겠어      1404.0
감정
0    2000
1    2000
2    2000
3    2000
4    2000
5    2000
dtype: int64


In [ ]:
# Train Data Labels
train_data["감정"] = train_data["감정"].astype('category')
train_data["emotion_label"] = train_data["감정"].cat.codes
train_features, train_labels = train_data['문장'], tf.one_hot(train_data["emotion_label"], 6)

# Test Data Labels
test_data["감정"] = test_data["감정"].astype('category')
test_data["emotion_label"] = test_data["감정"].cat.codes
test_features, test_labels = test_data['문장'], tf.one_hot(test_data["emotion_label"], 6)


      감정                        문장  Unnamed: 0  emotion_label
65226  2        열심히 회복해서 얼른 퇴원해야겠어     14799.0              2
65591  2           집에서 기분 전환을 해야겠어     15193.0              2
70106  1  최근에 살이 많이 쪄서 스트레스를 받고 있어      2121.0              1
84400  1              건강 검진을 받아야겠어     16882.0              1


In [ ]:
#tokenizing

mecab = Mecab() 

train_data['tokenized'] = train_data['문장'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_data['tokenized'] = test_data['문장'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

print(train_data.sample(n=4, random_state=1))

      감정                        문장  Unnamed: 0  emotion_label  \
65226  2        열심히 회복해서 얼른 퇴원해야겠어     14799.0              2   
65591  2           집에서 기분 전환을 해야겠어     15193.0              2   
70106  1  최근에 살이 많이 쪄서 스트레스를 받고 있어      2121.0              1   
84400  1              건강 검진을 받아야겠어     16882.0              1   

                             tokenized  
65226  [열심히, 회복, 해서, 얼른, 퇴원, 해야, 겠, 어]  
65591        [집, 에서, 기분, 전환, 해야, 겠, 어]  
70106   [최근, 살, 많이, 쪄서, 스트레스, 받, 있, 어]  
84400              [건강, 검진, 받, 아야, 겠어]  


In [ ]:
#tokenized seq-> X

X_train = train_data['tokenized'].values
X_test= test_data['tokenized'].values

In [ ]:
#max length for LSTM

max_len = 35

In [ ]:
#F1-score

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
#model fitting

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 2
total_cnt = len(tokenizer.word_index) #word count
rare_cnt = 0 #count word; frequency of appearance < threshold
total_freq = 0 #sum freq in all train data
rare_freq = 0 #sum freq in train data; frequency of appearance < threshold

#(word, frequency) ->key and value
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

vocab_size = total_cnt - rare_cnt + 2

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM
model.add(Dense(6, activation='softmax'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',
              metrics=['acc',tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), get_f1])
history = model.fit(X_train, train_labels, epochs=15, callbacks=[es, mc], batch_size=128, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1235400   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              234496    
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 1542      
                                                                 
Total params: 1,471,438
Trainable params: 1,471,438
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
638/638 [==============================] - ETA: 0s - loss: 1.2204 - acc: 0.4983 - recall: 0.2730 - precision: 0.7727 - get_f1: 0.3851
Epoch 1: val_acc improved from -inf to 0.02745, saving model to best_model.h5
638/638 [============

In [ ]:
loaded_model = load_model('best_model.h5', custom_objects={'get_f1' : get_f1})
loaded_model.evaluate(X_test, test_labels)

375/375 [==============================] - 3s 5ms/step - loss: 3.0792 - acc: 0.4712 - recall: 0.3529 - precision: 0.6465 - get_f1: 0.4345


[3.0791754722595215,
 0.4712499976158142,
 0.3529166579246521,
 0.6464661955833435,
 0.4345482289791107]

In [ ]:
def model_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  result = (loaded_model.predict(pad_new)>0.5).astype("int32").tolist()
  if 1 in result[0]:
    return result[0].index(1)
  else:
    return 6

In [ ]:
predict = [] #총 예측 결과
True_list = [] #실제 결과

for _, data in test_data.iterrows():
  True_list.append(data['감정'])
  p = model_predict(data['문장'])
  predict.append(p)

print(True_list)
print(predict)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 19ms/step
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

target_names = ['a', 'a', 'e', 'h', 'h', 's', 'none']

print(confusion_matrix(True_list, predict))
print(classification_report(True_list, predict, target_names=target_names))

[[ 790   87   78   36   86    0  923]
 [  84  779   99   71   63    0  904]
 [ 116  113  665   88   99    0  919]
 [  27   44   21 1389   13    0  506]
 [ 111   78  109   86  612    0 1004]
 [ 140  127  139   98  303    0 1193]
 [   0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

           a       0.62      0.40      0.48      2000
           a       0.63      0.39      0.48      2000
           e       0.60      0.33      0.43      2000
           h       0.79      0.69      0.74      2000
           h       0.52      0.31      0.39      2000
           s       0.00      0.00      0.00      2000
        none       0.00      0.00      0.00         0

    accuracy                           0.35     12000
   macro avg       0.45      0.30      0.36     12000
weighted avg       0.53      0.35      0.42     12000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1